<a href="https://colab.research.google.com/github/awais546/Intelligent-Cyber-Secuirty/blob/master/Copy_of_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

- If you are using Google Colab, you can simply install PySyft using the following command:
`! pip install syft`

- If you are using PySyft locally, the easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [3]:
import syft

In [2]:
! pip install syft

In [0]:
import torch as th
import syft as sy

In [0]:
hook = sy.TorchHook(th)  # this line of code creates an instance of th with its backend APIs modified with Syft functions

In [0]:
x = th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same
x

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [0]:
print(f'Type: {type(bob._objects)} \nValue: {bob._objects}')

Type: <class 'dict'> 
Value: {}


In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x =  x.send(bob) # send this data to bob

In [0]:
bob._objects

{77177638479: tensor([1, 2, 3, 4, 5])}

In [0]:
# What's the type of the pointer? and wht's the reason behind this type?
# What's its value?

print(f'Type: {type(x)} \nValue: {x}')

Type: <class 'torch.Tensor'> 
Value: (Wrapper)>[PointerTensor | me:4264420993 -> bob:77177638479]


In [0]:
x.location # where the Tensor located?

<VirtualWorker id:bob #objects:1>

In [0]:
x.id # the pointer ID at our machine

4264420993

In [0]:
x.id_at_location # the ID of the tensor at the remote worker

77177638479

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id='alice')

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2,3,4])
x = x.send(bob, alice)

In [0]:
# notice what does the send fucntion on two workers return; a multi-pointer
x

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:82003206698 -> bob:461181410]
	-> [PointerTensor | me:44384831308 -> alice:73102240321]

In [0]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 
print(f'Type: {type(x.child)} \nValue: {x.child}')
print(f'Type: {type(x.child.child)} \nValue: {x.child.child}')

Type: <class 'syft.generic.pointers.multi_pointer.MultiPointerTensor'> 
Value: [MultiPointerTensor]
	-> [PointerTensor | me:82003206698 -> bob:461181410]
	-> [PointerTensor | me:44384831308 -> alice:73102240321]
Type: <class 'dict'> 
Value: {'bob': [PointerTensor | me:82003206698 -> bob:461181410], 'alice': [PointerTensor | me:44384831308 -> alice:73102240321]}


In [0]:
bob._objects

{461181410: tensor([2, 3, 4])}

In [0]:
# try the .get() on the pointer
x = x.get()

x

[tensor([2, 3, 4]), tensor([2, 3, 4])]

In [0]:
alice._objects

{}

In [0]:
bob._objects

{}

In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice
x = th.tensor([2, 5, 4])
x = x.send(bob, alice)

In [0]:
# try the .get(sum_results=True) on your pointer
sum = x.get(sum_results=True)
sum

tensor([ 4, 10,  8])

In [0]:
bob._objects

{}

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x # a pointer to  a remote tensor located at bob

(Wrapper)>[PointerTensor | me:82718621241 -> bob:22533353205]

In [0]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:22940498579 -> bob:49348521690]

In [0]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:9663553913 -> bob:54125845716]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

<VirtualWorker id:bob #objects:0>

In [0]:
# create some toy data for our model
input_data = th.tensor([[1., 1],[0.5, 1],[1, 0],[0, 0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)


In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)


In [0]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

(Wrapper)>[PointerTensor | me:74328241521 -> bob:74691693238]

In [0]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()

  weights.data.sub_(weights.grad * lr)
  
  weights.grad *= 0

  print(loss.get().data)

tensor(0.4827)
tensor(0.3459)
tensor(0.2529)
tensor(0.1893)
tensor(0.1455)
tensor(0.1150)
tensor(0.0935)
tensor(0.0781)
tensor(0.0668)
tensor(0.0584)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{81360075755: tensor([1, 2, 3, 4, 5])}

In [0]:
del x  # delete the pointer to the remote object

In [0]:
bob._objects

{}

In [0]:
x.child.garbage_collect_data  # True by default

NameError: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{50328242358: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:58365664695 -> bob:37391219337]

In [0]:
bob._objects

{37391219337: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects  # some error from Jupyter is casuing this to appear!

{37391219337: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{37391219337: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects # notice that there is only a single tensor in bob

{25192765032: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
z = x + y

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [14]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).mean()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.1625)
tensor(0.7299)
tensor(0.5157)
tensor(0.4038)
tensor(0.3404)
tensor(0.3003)
tensor(0.2718)
tensor(0.2494)
tensor(0.2305)
tensor(0.2138)


# nothing was federated up to this point!

In [15]:
# let's reeat the previous experiment in a FL approach:

import torch as th
import syft as sy

hook = sy.TorchHook(th)  

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

bob = bob.clear_objects()
alice = alice.clear_objects()

In [0]:
# create local datasets at Bob and Alice
data_bob = th.tensor([[1.,1],[0,1]], requires_grad=True).send(bob)
target_bob = th.tensor([[1.],[1]], requires_grad=True).send(bob)

data_alice = th.tensor([[1., 0],[0, 0]], requires_grad=True).send(alice)
target_alice = th.tensor([[0.],[0]], requires_grad=True).send(alice)

In [17]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:74417622266 -> bob:51537680643],
  (Wrapper)>[PointerTensor | me:96932882400 -> bob:58232608014]),
 ((Wrapper)>[PointerTensor | me:58861197680 -> alice:27530023612],
  (Wrapper)>[PointerTensor | me:99696396379 -> alice:25482168615])]

In [0]:
def train(iterations=3):

    model =  nn.Linear(2,1) # our model created locally
    # our optimizer created locally
    opt = optim.SGD(params=model.parameters(), lr=0.1) 
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)
            opt.zero_grad()

            pred = model(_data)

            loss = ((pred - _target)**2).mean()

            loss.backward()

            opt.step()
            
            model = model.get()
            # do training on the remote machine
            # 1 zero the gradients

            # 2 calculate predictions

            # 3 calculate loss -- MSE

            # 4 calculate gradeints

            # 5 update weights

            # return the model to the local machine
            
            
        print(loss.get())

In [19]:
train(3)

tensor(0.0903, requires_grad=True)
tensor(0.0128, requires_grad=True)
tensor(0.0053, requires_grad=True)


# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [20]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [18]:
bob._objects

{67129206719: tensor([1, 2, 3, 4, 5])}

In [19]:
alice._objects

{47619719782: (Wrapper)>[PointerTensor | alice:47619719782 -> bob:67129206719]}

In [0]:
y = x + x

In [21]:
y

(Wrapper)>[PointerTensor | me:14936897555 -> alice:64694922675]

In [22]:
bob._objects

{62582929614: tensor([ 2,  4,  6,  8, 10]),
 67129206719: tensor([1, 2, 3, 4, 5])}

In [23]:
alice._objects

{47619719782: (Wrapper)>[PointerTensor | alice:47619719782 -> bob:67129206719],
 64694922675: (Wrapper)>[PointerTensor | alice:64694922675 -> bob:62582929614],
 86345919204: (Wrapper)>[PointerTensor | alice:64694922675 -> bob:62582929614]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [22]:
bob.clear_objects()
alice.clear_objects()
jon.clear_objects()



<VirtualWorker id:jon #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [27]:
z = x + y


TensorsNotCollocatedException: ignored

In [28]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:45237475009 -> bob:15381591198]

In [29]:
bob._objects

{9094970848: tensor([1, 2, 3, 4, 5]), 15381591198: tensor([1, 2, 3, 4, 5])}

In [30]:
alice._objects

{}

In [31]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [32]:
bob._objects

{9094970848: tensor([1, 2, 3, 4, 5])}

# Lesson: Pointer Chain Operations

In [54]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [35]:
bob._objects

{38747914675: tensor([1, 2, 3, 4, 5])}

In [36]:
alice._objects

{}

In [37]:
x.move(alice)

(Wrapper)>[PointerTensor | me:97050578686 -> alice:57396779018]

In [38]:
x

(Wrapper)>[PointerTensor | me:97050578686 -> alice:57396779018]

In [39]:
bob._objects

{}

In [40]:
alice._objects

{57396779018: tensor([1, 2, 3, 4, 5])}

In [41]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [43]:
bob._objects

{70335146950: tensor([1, 2, 3, 4, 5])}

In [44]:
alice._objects

{71496741233: (Wrapper)>[PointerTensor | alice:71496741233 -> bob:70335146950]}

In [45]:
x

(Wrapper)>[PointerTensor | me:17026547061 -> alice:71496741233]

In [46]:
x.remote_get() # inplace operation

(Wrapper)>[PointerTensor | me:17026547061 -> alice:71496741233]

In [47]:
bob._objects

{}

In [48]:
alice._objects

{71496741233: tensor([1, 2, 3, 4, 5])}

In [49]:
x.move(bob)

(Wrapper)>[PointerTensor | me:72210611048 -> bob:17026547061]

In [50]:
x

(Wrapper)>[PointerTensor | me:72210611048 -> bob:17026547061]

In [51]:
bob._objects

{17026547061: tensor([1, 2, 3, 4, 5])}

In [52]:
alice._objects

{}


**Exercise:**

To avoid exposing gradients among participants, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [0]:
import syft as sy
import torch as th
from torch import nn, optim

In [0]:
# create workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")


In [81]:
bob.clear_objects()
alice.clear_objects()
secureWorker.clear_objects()

<VirtualWorker id:secureWorker #objects:0>

In [82]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])

<VirtualWorker id:secureWorker #objects:0>

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True) 

In [0]:
# create local datasets at Bob and Alice

data_bob = th.tensor([[1.,1],[0,1]], requires_grad=True).send(bob)
target_bob = th.tensor([[1.],[1]], requires_grad=True).send(bob)

data_alice = th.tensor([[1., 0],[0, 0]], requires_grad=True).send(alice)
target_alice = th.tensor([[0.],[0]], requires_grad=True).send(alice)

In [0]:
# create a linear model at local worker
model = nn.Linear(2,1)

In [0]:
# Send copies of the linear model to alice and bob
model_bob = model.copy().send(bob)
model_alice = model.copy().send(alice)

In [0]:
# create two opimizers for Alice and Bob
opt_bob = optim.SGD(params=model_bob.parameters(), lr=0.1) 
opt_alice = optim.SGD(params=model_alice.parameters(), lr=0.1) 

In [0]:

# train the models at alice and bob inside one loop

for i in range(2):

            # send model to the data
            #model = model.send(_data.location)
            opt_bob.zero_grad()
            opt_alice.zero_grad()

            pred_bob = model_bob(data_bob)
            pred_alice = model_alice(data_alice)

            loss_bob = ((pred_bob - target_bob)**2).mean()
            loss_alice = ((pred_alice - target_alice)**2).mean()

            loss_bob.backward()
            loss_alice.backward()

            opt_bob.step()
            opt_alice.step()
            



In [0]:
awais = sy.VirtualWorker(hook, id="awais")


In [0]:
model_bob.move(awais)


In [0]:
model_alice.move(awais)

In [159]:
import torch
weights = th.tensor([0,0])
bias = th.tensor(0)



tensor(0)


In [0]:
for each in awais._objects:
  if(len(awais._objects[each][0])) == len(weights):
    weights = weights+awais._objects[each][0]
  else:
    bias = bias+awais._objects[each]

In [161]:
print(weights)
print(bias)

tensor([0.0049, 0.8588], grad_fn=<AddBackward0>)
tensor([0.5183], grad_fn=<AddBackward0>)


In [0]:
awais_model = model.copy().send(awais)

In [0]:
awais_model.weight.data = weights
awais_model.bias.data = bias

In [174]:
model = awais_model.get()

AttributeError: ignored

In [0]:

# move the models to the third worker 
# aggreegate the models (their average)
# --- use model.weight.data to access the weights, and model.bias.data to access bias
# --- use model.weight.set_(new_weights) to update the weights
# --- use model.bias.set_(new_bias) to update the bias

In [0]:
# send the model back to the local worker

In [0]:
# make predictions uding the aggregated model